In [1]:
#https://www.huduser.gov/portal/datasets/fmr.html#data_2024

import pandas as pd
import numpy as np

In [2]:
# Import raw data
df=pd.read_csv("Rent Data 2017-2024.csv" ,sep="," ,encoding="utf-8")
df.head()

,Year,stusps,state,hud_area_code,countyname,county_town_name,metro,hud_area_name,fips,pop2020,fmr_0,fmr_1,fmr_2,fmr_3,fmr_4
0,2024,AL,1,METRO33860M33860,Autauga County,NaN,1.0,"Montgomery, AL MSA",100199999,55639.0,836,913,1092,1383,1753
1,2024,AL,1,METRO19300M19300,Baldwin County,NaN,1.0,"Daphne-Fairhope-Foley, AL MSA",100399999,218289.0,1051,1056,1362,1670,2114
2,2024,AL,1,NCNTY01005N01005,Barbour County,NaN,0.0,"Barbour County, AL",100599999,25026.0,652,656,857,1089,1141
3,2024,AL,1,METRO13820M13820,Bibb County,NaN,1.0,"Birmingham-Hoover, AL HUD Metro FMR Area",100799999,22374.0,983,1109,1245,1570,1752
4,2024,AL,1,METRO13820M13820,Blount County,NaN,1.0,"Birmingham-Hoover, AL HUD Metro FMR Area",100999999,57755.0,983,1109,1245,1570,1752


In [3]:
# Clean raw data
df = df.rename(columns={'stusps': 'State', 'state': 'State No.','countyname':'County','pop2020':'Population','fmr_0':"Studio","fmr_1":"One Bedroom","fmr_2":"Two Bedroom","fmr_3":"Three Bedroom","fmr_4":"Four Bedroom"})
df = df[['Year','State', 'County','Population','Studio','One Bedroom','Two Bedroom','Three Bedroom','Four Bedroom']]

df = df.dropna()

df["Year"] = df['Year'].astype(int)
df["Studio"] = df['Studio'].astype(float)
df["One Bedroom"] = df['One Bedroom'].astype(float)
df["Two Bedroom"] = df['Two Bedroom'].astype(float)
df["Three Bedroom"] = df['Three Bedroom'].astype(float)
df["Four Bedroom"] = df['Four Bedroom'].astype(float)
df["Population"] = df['Population'].astype(int)

df = df[~df['State'].isin(['AS','GU','MP','PR', 'VI'])]

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37454 entries, 0 to 38045
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Year           37454 non-null  int32  
 1   State          37454 non-null  object 
 2   County         37454 non-null  object 
 3   Population     37454 non-null  int32  
 4   Studio         37454 non-null  float64
 5   One Bedroom    37454 non-null  float64
 6   Two Bedroom    37454 non-null  float64
 7   Three Bedroom  37454 non-null  float64
 8   Four Bedroom   37454 non-null  float64
dtypes: float64(5), int32(2), object(2)
memory usage: 2.6+ MB


In [4]:
# Save cleaned file
df.to_csv('clean_rent_data.csv', index=True, index_label='Index')

In [5]:


# Drop the 'County' column before grouping by 'State' and 'Year'
dfdrop = df.drop('County', axis=1)

# Group by State and Year to calculate average rent
average_prices = dfdrop.groupby(['State', 'Year']).mean().reset_index()

average_prices = average_prices.sort_values(by=['State', 'Year'])

# Calculate the percentage change for each type of apartment rent within each state
average_prices['Studio_Percent_Change'] = average_prices.groupby('State')['Studio'].pct_change() * 100
average_prices['One_Bedroom_Percent_Change'] = average_prices.groupby('State')['One Bedroom'].pct_change() * 100
average_prices['Two_Bedroom_Percent_Change'] = average_prices.groupby('State')['Two Bedroom'].pct_change() * 100
average_prices['Three_Bedroom_Percent_Change'] = average_prices.groupby('State')['Three Bedroom'].pct_change() * 100
average_prices['Four_Bedroom_Percent_Change'] = average_prices.groupby('State')['Four Bedroom'].pct_change() * 100

# Drop NaN values (first year for each state)
average_prices = average_prices.dropna()

# Save the cleaned DataFrame to a CSV file
average_prices.to_csv('clean_rent_percentages.csv', index=True, index_label='Index')

In [6]:
# Drop the 'County' column before grouping by 'State' and 'Year'

# Group by State and Year to calculate average rent
average_prices = dfdrop.groupby(['State', 'Year']).mean().reset_index()

average_prices = average_prices.sort_values(by=['State', 'Year'])

# Calculate the dollar change for each type of apartment rent within each state
average_prices['Studio_Dollar_Change'] = average_prices.groupby('State')['Studio'].diff()
average_prices['One_Bedroom_Dollar_Change'] = average_prices.groupby('State')['One Bedroom'].diff()
average_prices['Two_Bedroom_Dollar_Change'] = average_prices.groupby('State')['Two Bedroom'].diff()
average_prices['Three_Bedroom_Dollar_Change'] = average_prices.groupby('State')['Three Bedroom'].diff()
average_prices['Four_Bedroom_Dollar_Change'] = average_prices.groupby('State')['Four Bedroom'].diff()

# Drop NaN values (first year for each state)
average_prices = average_prices.dropna()

# Group by State and sum the dollar changes
sum_changes_by_state = average_prices.groupby('State').agg({
    'Studio_Dollar_Change': 'sum',
    'One_Bedroom_Dollar_Change': 'sum',
    'Two_Bedroom_Dollar_Change': 'sum',
    'Three_Bedroom_Dollar_Change': 'sum',
    'Four_Bedroom_Dollar_Change': 'sum'
}).reset_index()
# Drop the 'County' column before grouping by 'State' and 'Year'
#sum_changes_by_state = sum_changes_by_state.drop(['Population','Studio','One Bedroom','Two Bedroom','Three Bedroom','Four Bedroom'], axis=1)
# Save the cleaned DataFrame to a CSV file
sum_changes_by_state.to_csv('clean_rent_dollars.csv', index=True, index_label='Index')

In [17]:
# Calculate rent per capita for each bedroom type
# Drop rows where Population is 0
df = df[df['Population'] > 1000000]

df['Rent Per Capita Studio'] = df['Studio'] / df['Population']
df['Rent Per Capita One Bedroom'] = df['One Bedroom'] / df['Population']
df['Rent Per Capita Two Bedroom'] = df['Two Bedroom'] / df['Population']
df['Rent Per Capita Three Bedroom'] = df['Three Bedroom'] / df['Population']
df['Rent Per Capita Four Bedroom'] = df['Four Bedroom'] / df['Population']

df = df.dropna()
df_sorted = df.sort_values(by='Rent Per Capita Studio', ascending=True)

df_sorted.head(10)

,Year,State,County,Population,Studio,One Bedroom,Two Bedroom,Three Bedroom,Four Bedroom,Rent Per Capita Studio,Rent Per Capita One Bedroom,Rent Per Capita Two Bedroom,Rent Per Capita Three Bedroom,Rent Per Capita Four Bedroom
33565,2017,CA,Los Angeles County,9818605,988.0,1195.0,1545.0,2079.0,2303.0,0.000101,0.000122,0.000157,0.000212,0.000235
28796,2018,CA,Los Angeles County,9818605,1067.0,1284.0,1663.0,2231.0,2467.0,0.000109,0.000131,0.000169,0.000227,0.000251
24029,2019,CA,Los Angeles County,9818605,1158.0,1384.0,1791.0,2401.0,2641.0,0.000118,0.000141,0.000182,0.000245,0.000269
9732,2020,CA,Los Angeles County,10105720,1279.0,1517.0,1956.0,2614.0,2857.0,0.000127,0.000150,0.000194,0.000259,0.000283
14498,2021,CA,Los Angeles County,10105720,1369.0,1605.0,2058.0,2735.0,2982.0,0.000135,0.000159,0.000204,0.000271,0.000295
19264,2022,CA,Los Angeles County,10105720,1384.0,1604.0,2044.0,2693.0,2933.0,0.000137,0.000159,0.000202,0.000266,0.000290
4969,2023,CA,Los Angeles County,10040682,1534.0,1747.0,2222.0,2888.0,3170.0,0.000153,0.000174,0.000221,0.000288,0.000316
33464,2017,AZ,Maricopa County,3817117,624.0,757.0,944.0,1374.0,1594.0,0.000163,0.000198,0.000247,0.000360,0.000418
29363,2018,IL,Cook County,5194675,879.0,1014.0,1180.0,1501.0,1794.0,0.000169,0.000195,0.000227,0.000289,0.000345
37285,2017,TX,Harris County,4092459,701.0,797.0,976.0,1333.0,1690.0,0.000171,0.000195,0.000238,0.000326,0.000413
